# 对话聊天

使用一个大型语言模型的一个令人兴奋的事情是，我们可以用它来构建一个定制的聊天机器人，只需要很少的工作量。在这一节中，我们将探索如何利用聊天格式（接口）与个性化或专门针对特定任务或行为的聊天机器人进行延伸对话。


## 启动

In [1]:
import os
import openai

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

像 ChatGPT 这样的聊天模型实际上是组装成以一系列消息作为输入，并返回一个模型生成的消息作为输出的。虽然聊天格式的设计旨在使这种多轮对话变得容易，但我们通过之前的学习可以知道，它对于没有任何对话的单轮任务也同样有用。

接下来，我们将定义两个辅助函数。第一个是单轮的，我们将prompt放入看起来像是某种用户消息的东西中。另一个则传入一个消息列表。这些消息可以来自不同的角色，我们会描述一下这些角色。

第一条消息是一个系统消息，它提供了一个总体的指示，然后在这个消息之后，我们有用户和助手之间的交替。如果你曾经使用过 ChatGPT 网页界面，那么你的消息是用户消息，而 ChatGPT 的消息是助手消息。系统消息则有助于设置助手的行为和角色，并作为对话的高级指示。你可以想象它在助手的耳边低语，引导它的回应，而用户不会注意到系统消息。

因此，作为用户，如果你曾经使用过 ChatGPT，你可能不知道 ChatGPT 的系统消息是什么，这是有意为之的。系统消息的好处是为开发者提供了一种方法，在不让请求本身成为对话的一部分的情况下，引导助手并指导其回应。

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 控制模型输出的随机程度
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 控制模型输出的随机程度
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

现在让我们尝试在对话中使用这些消息。我们将使用上面的函数来获取从这些消息中得到的回答，同时，使用更高的 temperature（越高生成的越多样）。

系统消息说，你是一个说话像莎士比亚的助手。这是我们向助手描述它应该如何表现的方式。然后，第一个用户消息是，给我讲个笑话。接下来的消息是，为什么鸡会过马路？然后最后一个用户消息是，我不知道。

In [4]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, fair sir.


In [3]:
# 中文
messages =  [  
{'role':'system', 'content':'你是一个像莎士比亚一样说话的助手。'},    
{'role':'user', 'content':'给我讲个笑话'},   
{'role':'assistant', 'content':'鸡为什么过马路'},   
{'role':'user', 'content':'我不知道'}  ]

In [9]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

因为它要去到另一边！哈哈哈！


让我们做另一个例子。助手的消息是，你是一个友好的聊天机器人，第一个用户消息是，嗨，我叫Isa。我们想要得到第一个用户消息。

In [10]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's great to meet you. How can I assist you today?


In [11]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'Hi, 我是Isa。'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

嗨，Isa，很高兴见到你！有什么我可以帮助你的吗？


让我们再试一个例子。系统消息是，你是一个友好的聊天机器人，第一个用户消息是，是的，你能提醒我我的名字是什么吗？

In [12]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but since we don't have any personal information about you, I don't know your name. Can you please tell me your name?


In [14]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'好，你能提醒我，我的名字是什么吗？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

抱歉，我不知道您的名字，因为我们是虚拟的聊天机器人和现实生活中的人类在不同的世界中。


如上所见，模型实际上并不知道我的名字。

因此，每次与语言模型的交互都是一个独立的交互，这意味着我们必须提供所有相关的消息，以便模型在当前对话中进行引用。如果想让模型引用或 “记住” 对话的早期部分，则必须在模型的输入中提供早期的交流。我们将其称为上下文。让我们试试。

In [15]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


In [16]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是Isa'},
{'role':'assistant', 'content': "Hi Isa! 很高兴认识你。今天有什么可以帮到你的吗?"},
{'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

当然可以！您的名字是Isa。


现在我们已经给模型提供了上下文，也就是之前的对话中提到的我的名字，然后我们会问同样的问题，也就是我的名字是什么。因为模型有了需要的全部上下文，所以它能够做出回应，就像我们在输入的消息列表中看到的一样。

# 订餐机器人

现在，我们构建一个 “订餐机器人”，我们需要它自动收集用户信息，接受比萨饼店的订单。

下面这个函数将收集我们的用户消息，以便我们可以避免手动输入，就像我们在刚刚上面做的那样。这个函数将从我们下面构建的用户界面中收集提示，然后将其附加到一个名为上下文的列表中，并在每次调用模型时使用该上下文。模型的响应也会被添加到上下文中，所以模型消息和用户消息都被添加到上下文中，因此上下文逐渐变长。这样，模型就有了需要的信息来确定下一步要做什么。

In [3]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

现在，我们将设置并运行这个 UI 来显示订单机器人。初始的上下文包含了包含菜单的系统消息。请注意，上下文会随着时间的推移而不断增长。

In [5]:
%pip install --upgrade panel

  Using cached panel-0.14.4-py2.py3-none-any.whl (20.8 MB)
  Using cached bokeh-2.4.3-py3-none-any.whl (18.5 MB)
  Using cached param-1.13.0-py2.py3-none-any.whl (87 kB)
  Using cached pyviz_comms-2.2.1-py2.py3-none-any.whl (42 kB)
                                              0.0/93.9 kB ? eta -:--:--
     -------------                            30.7/93.9 kB 1.4 MB/s eta 0:00:01
     ------------------------               61.4/93.9 kB 656.4 kB/s eta 0:00:01
     -------------------------------------- 93.9/93.9 kB 767.6 kB/s eta 0:00:00
  Using cached pyct-0.5.0-py2.py3-none-any.whl (15 kB)
  Using cached bleach-6.0.0-py3-none-any.whl (162 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --upgrade openai

     -------------------------------------- 71.9/71.9 kB 655.9 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.26.5
    Uninstalling openai-0.26.5:
      Successfully uninstalled openai-0.26.5
Note: you may need to restart the kernel to use updated packages.


In [6]:
import bokeh.models
from bokeh.models import Box

In [12]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

In [11]:
%pip install --upgrade bokeh

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install jupyter_bokeh

  Using cached jupyter_bokeh-3.0.7-py3-none-any.whl (1.4 MB)
  Using cached bokeh-3.1.1-py3-none-any.whl (8.3 MB)
  Using cached ipywidgets-8.0.6-py3-none-any.whl (138 kB)
  Using cached contourpy-1.0.7-cp38-cp38-win_amd64.whl (162 kB)
  Using cached xyzservices-2023.2.0-py3-none-any.whl (55 kB)
  Using cached widgetsnbextension-4.0.7-py3-none-any.whl (2.1 MB)
  Using cached jupyterlab_widgets-3.0.7-py3-none-any.whl (198 kB)
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.4.3
    Uninstalling bokeh-2.4.3:
      Successfully uninstalled bokeh-2.4.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.14.4 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.1.1 which is incompatible.


In [13]:
dashboard

ImportError: cannot import name 'warn' from 'bokeh.util.warnings' (c:\PythonVenv\my_project_venv\lib\site-packages\bokeh\util\warnings.py)

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

现在我们可以要求模型创建一个 JSON 摘要发送给订单系统。

所以我们现在追加另一个系统消息，它是另一条prompt，我们说创建一个刚刚订单的 JSON 摘要，列出每个项目的价格，字段应包括1）披萨，包括尺寸，2）配料列表，3）饮料列表，4）辅菜列表，包括尺寸，最后是总价格。这里也可以在这里使用用户消息，不一定是系统消息。

请注意，这里我们使用了一个较低的temperature，因为对于这些类型的任务，我们希望输出相对可预测。

In [45]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "cheese",
    "size": "large",
    "toppings": [
      "mushrooms"
    ],
    "price": 12.45
  },
  "drinks": [
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    },
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    }
  ],
  "sides": [],
  "total_price": 18.45
}
``` 

Note: I assumed that the price of the large cheese pizza with mushrooms is $12.45 instead of $12.95, since the customer only ordered one topping.


In [15]:
# 中文
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{'role':'system', 'content':"""
你是订餐机器人，为披萨餐厅自动收集订单信息。
你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。
最后需要询问是否自取或外送，如果是外送，你要询问地址。
最后告诉顾客订单总金额，并送上祝福。

请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。

菜单包括：

菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

In [17]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'848d3414-6b6c-44fb-8890-6ce800fcaca7': {'defs': …

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'创建上一个食品订单的 json 摘要。\
逐项列出每件商品的价格，字段应该是 1) 披萨，包括大小 2) 配料列表 3) 饮料列表，包括大小 4) 配菜列表包括大小 5) 总价'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

现在，我们已经建立了自己的订餐聊天机器人。请随意自定义并修改系统消息，以更改聊天机器人的行为，并使其扮演不同的角色和拥有不同的知识。

## 尝试你的实验！

你可以修改菜单或指令来创建自己的订单机器人！